In [ ]:
import socket
import threading

# Pre-approved user credentials
ACCOUNTS = {
    "alice": "apple123",
    "bob": "banana456",
    "root": "toor987"
}

# Authenticated clients: { (ip, port): username }
active_clients = {}

# Caesar Cipher: Shifts ASCII values
def caesar_encrypt(text, shift=4):
    return ''.join(chr((ord(c) - 32 + shift) % 95 + 32) if c.isprintable() else c for c in text)

def caesar_decrypt(text, shift=4):
    return ''.join(chr((ord(c) - 32 - shift) % 95 + 32) if c.isprintable() else c for c in text)

# Main listener for handling all incoming data
def chat_listener(sock):
    while True:
        try:
            msg_data, client = sock.recvfrom(2048)
            text = msg_data.decode()

            # Handle new login attempt
            if client not in active_clients:
                if ":" in text:
                    uname, pwd = text.strip().split(":", 1)
                    if uname in ACCOUNTS and ACCOUNTS[uname] == pwd:
                        active_clients[client] = uname
                        sock.sendto("OK".encode(), client)
                        print(f"[+] {uname} logged in from {client}")
                    else:
                        sock.sendto("FAIL".encode(), client)
                        print(f"[-] Invalid login from {client}")
                else:
                    sock.sendto("FAIL".encode(), client)
                continue

            # Decrypt, log, then rebroadcast
            sender_name = active_clients[client]
            plain_msg = caesar_decrypt(text)
            print(f"[{sender_name}@{client}]: {plain_msg}")

            rebroadcast = caesar_encrypt(f"{sender_name}: {plain_msg}")
            for peer in active_clients:
                if peer != client:
                    sock.sendto(rebroadcast.encode(), peer)

        except Exception as err:
            print(f"[!] Error from {client}: {err}")

# Boot the server
def launch_chat_server():
    srv = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    srv.bind(("127.0.0.1", 5000))
    print("UDP Chat Server is live on port 5000.")

    thread = threading.Thread(target=chat_listener, args=(srv,))
    thread.start()

    try:
        thread.join()
    except KeyboardInterrupt:
        print("\n[!] Server shutting down...")
    finally:
        srv.close()

if __name__ == "__main__":
    launch_chat_server()
